# Open Precinct Data Schema 0.1

_Michal Migurski, December 2018_

This is a preliminary exploration of a data format for storing U.S. electoral precincts based on the argument in [_Open Precinct Data_ (April 2018)](https://medium.com/planscore/open-precinct-data-eec479287715): “Imagine if you could easily correlate detailed voting results from OpenElections.net (OE) or state boards of elections with mapped polygons and census geography over time.”

The sample data here covers a small part of North Carolina. `shapes.shp` includes geometries for Congressional districts with 2012, 2014, and 2016 borders, [Johnston](https://en.wikipedia.org/wiki/Johnston_County,_North_Carolina) and [Alamance counties](https://en.wikipedia.org/wiki/Alamance_County,_North_Carolina), and all their voting precincts. U.S. House candidates for each included district over three general elections are included. The data is also [available in Google Docs for easier browsing](https://docs.google.com/spreadsheets/d/1griVfVeBYahOOd4UoZds6R1NXga_QGWbAZXRcJ0OvJY).

This schema is based on the ecosystem around GTFS, the successful data standard for transit schedule data pioneered by Portland TriMet and Google Maps over ten years ago. Some design goals include:

- Ease of use by data scientists and political scientists, whose tools include Python or R notebooks, spreadsheet applications, or GIS software.
- Accessibility to parsing software similar to [Partridge by Danny Whalen and Remix](https://github.com/remix/partridge).
- Compatibility with parallel electoral data projects such as [Open Elections](http://www.openelections.net).

![Content of shapes.shp](NC-counties.png)

## Loading Data

Open Precinct Data is stored in a zip file with five contained files:

- [`elections.csv`](https://docs.google.com/spreadsheets/d/1griVfVeBYahOOd4UoZds6R1NXga_QGWbAZXRcJ0OvJY/edit#gid=0) – one or more elections
- [`districts.csv`](https://docs.google.com/spreadsheets/d/1griVfVeBYahOOd4UoZds6R1NXga_QGWbAZXRcJ0OvJY/edit#gid=293778653) – electoral districts for each election
- [`candidates.csv`](https://docs.google.com/spreadsheets/d/1griVfVeBYahOOd4UoZds6R1NXga_QGWbAZXRcJ0OvJY/edit#gid=2053810651) – candidate details such as political party and incumbency for each district
- [`precincts.csv`](https://docs.google.com/spreadsheets/d/1griVfVeBYahOOd4UoZds6R1NXga_QGWbAZXRcJ0OvJY/edit#gid=386435423) – voting precincts where candidate votes are tallied for each election
- [`shapes.shp`](https://docs.google.com/spreadsheets/d/1griVfVeBYahOOd4UoZds6R1NXga_QGWbAZXRcJ0OvJY/edit#gid=2038374705) – geographic areas for precincts and districts

We start by loading data from each of these files into [Pandas DataFrames](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html), using [GeoPandas](http://geopandas.org) for shapes:

In [1]:
import geopandas, pandas, zipfile

with zipfile.ZipFile('OpenPrecincts-Example-0.1.zip') as zf:
    with zf.open('elections.csv') as f:
        elections = pandas.read_csv(f)
    with zf.open('districts.csv') as f:
        districts = pandas.read_csv(f)
    with zf.open('candidates.csv') as f:
        candidates = pandas.read_csv(f)
    with zf.open('precincts.csv') as f:
        precincts = pandas.read_csv(f)

shapes = geopandas.read_file('/vsizip/OpenPrecincts-Example-0.1.zip/shapes.shp')

## One Precinct

Let’s look at a single precinct. The term _precinct_ is used loosely in this project. A precinct is any geographic area where votes are counted. For example, absentee ballots for entire counties may be included in this list. Here, we select a single precinct covering [Haw River in Alamance County](https://en.wikipedia.org/wiki/Haw_River,_North_Carolina).

- `PSID:1158849879` is a unique, opaque identifier for geographic shapes in `shapes.shp`
- `OPID:001`, `OPID:002`, and `OPID:003` are unique, opaque identifiers for three elections in `elections.csv`
- `OPID:012`, `OPID:013`, and `OPID:016` are unique, opaque identifiers for three districts in `districts.csv`
- The precinct changes name over time, but it’s always the same geographic area
- For the first two elections the precinct covers two districts, `OPID:012` and `OPID:013`
- In the final election held after North Carolina’s 2016 redistricting, the precinct covers just one district, `OPID:016`
- Both `OPID:013` and `OPID:016` can be called “District 6” but each belongs to a different plan
- Identifiers like `OPID:nnn+` and `PSID:nnn+` are completely opaque and internal to Open Precinct Data
- Identifiers like `FIPS:nnn+` are [defined by the U.S. government](https://www.nist.gov/itl/itl-publications/federal-information-processing-standards-fips) and used by the Census

In [2]:
precincts[precincts.shape_id == 'PSID:1158849879']

,election_id,district_id,county_id,county_name,precinct_name,shape_id,(shape type),(district number)
36,OPID:001,OPID:012,FIPS:37001,ALAMANCE,13_HAW RIVER,PSID:1158849879,precinct,4
37,OPID:001,OPID:013,FIPS:37001,ALAMANCE,13_HAW RIVER,PSID:1158849879,precinct,6
122,OPID:002,OPID:012,FIPS:37001,ALAMANCE,13,PSID:1158849879,precinct,4
123,OPID:002,OPID:013,FIPS:37001,ALAMANCE,13,PSID:1158849879,precinct,6
196,OPID:003,OPID:016,FIPS:37001,ALAMANCE,13,PSID:1158849879,precinct,6


## Connecting Precincts to Elections

Our sample data includes two counties and three elections. Let’s look at candidate incumbency for an election in one county: 2014 in Alamance County. We start by matching on `elections.election_date` and `precincts.county_name`, and using [`pandas.merge()`](https://pandas.pydata.org/pandas-docs/stable/merging.html#database-style-dataframe-joining-merging) to select a subset of precincts.

In [3]:
alamance2014precincts = pandas.merge(
    elections[elections.election_date == '2014-11-04'],
    precincts[precincts.county_name == 'ALAMANCE'],
    on='election_id')

alamance2014precincts.head()

,election_id,state_id,election_date,election_type,district_id,county_id,county_name,precinct_name,shape_id,(shape type),(district number)
0,OPID:002,FIPS:37,2014-11-04,general,OPID:011,FIPS:37001,ALAMANCE,01,PSID:1158854937,precinct,2
1,OPID:002,FIPS:37,2014-11-04,general,OPID:013,FIPS:37001,ALAMANCE,02,PSID:1158848961,precinct,6
2,OPID:002,FIPS:37,2014-11-04,general,OPID:013,FIPS:37001,ALAMANCE,035,PSID:1158850573,precinct,6
3,OPID:002,FIPS:37,2014-11-04,general,OPID:013,FIPS:37001,ALAMANCE,03C,PSID:1158854377,precinct,6
4,OPID:002,FIPS:37,2014-11-04,general,OPID:013,FIPS:37001,ALAMANCE,03N,PSID:1158852503,precinct,6


Party incumbency is useful when predicting election outcomes: candidates running for re-election often have a track record and name recognition which can help them in the polls. We perform another `pandas.merge()` to connect our Alamance 2014 precincts to incumbent candidates, this time with a left join.

The complete list of 38 precincts has a mix of Republican and Democratic incumbents along with an empty seat. In 2014, three U.S. House districts overlapped with Alamance County.

In [4]:
alamance2014incumbents = pandas.merge(
    alamance2014precincts,
    candidates[candidates.candidate_incumbent == 'Yes'],
    on=('district_id', 'election_id'),
    how='left')

alamance2014incumbents[[
    'election_date', 'district_id', 'county_name',
    'precinct_name', 'candidate_party', 'shape_id']]

,election_date,district_id,county_name,precinct_name,candidate_party,shape_id
0,2014-11-04,OPID:011,ALAMANCE,01,Rep,PSID:1158854937
1,2014-11-04,OPID:013,ALAMANCE,02,NaN,PSID:1158848961
2,2014-11-04,OPID:013,ALAMANCE,035,NaN,PSID:1158850573
3,2014-11-04,OPID:013,ALAMANCE,03C,NaN,PSID:1158854377
4,2014-11-04,OPID:013,ALAMANCE,03N,NaN,PSID:1158852503
5,2014-11-04,OPID:013,ALAMANCE,03N2,NaN,PSID:1158850109
6,2014-11-04,OPID:013,ALAMANCE,03S,NaN,PSID:1158851219
7,2014-11-04,OPID:013,ALAMANCE,03W,NaN,PSID:1158851533
8,2014-11-04,OPID:013,ALAMANCE,04,NaN,PSID:1158849677
9,2014-11-04,OPID:013,ALAMANCE,05,NaN,PSID:1158850461


## Output to GIS

Finally, we can link the table above to geographic areas to see how this county election looks on a map. Both precincts and districts include a `shape_id` foreign key that we can find in `shapes.shp`. After merging, we use a [`geopandas.GeoSeries.intersection()`](http://geopandas.org/reference.html#geopandas.GeoSeries.intersection) to split multi-district precincts like Haw River (precinct 13) among their districts.

In [5]:
alamance_df1 = pandas.merge(
    alamance2014incumbents.merge(shapes, on='shape_id'),
    districts.merge(shapes, on='shape_id'),
    suffixes=('_P', '_D'),
    on='district_id')

alamance_df1['geometry'] = \
    geopandas.GeoSeries(alamance_df1.geometry_P)\
    .intersection(geopandas.GeoSeries(alamance_df1.geometry_D))

alamance_df2 = alamance_df1[[
    'election_date', 'district_name', 'county_name',
    'precinct_name', 'candidate_party', 'geometry']]

alamance_df2.head()

,election_date,district_name,county_name,precinct_name,candidate_party,geometry
0,2014-11-04,U.S. House District 2,ALAMANCE,01,Rep,POLYGON ((-79.43197308657012 35.88929191508172...
1,2014-11-04,U.S. House District 6,ALAMANCE,02,NaN,POLYGON ((-79.53483899133798 36.04382100018476...
2,2014-11-04,U.S. House District 6,ALAMANCE,035,NaN,POLYGON ((-79.50051299120325 36.15426500013933...
3,2014-11-04,U.S. House District 6,ALAMANCE,03C,NaN,"POLYGON ((-79.5203239911727 36.08219700003401,..."
4,2014-11-04,U.S. House District 6,ALAMANCE,03N,NaN,POLYGON ((-79.49082799096274 36.10568999961695...


Finally, we can see precinct 13 in the map below, with its Northern portion in [David Price’s](https://ballotpedia.org/David_Price) Democratic District 4 and its Southern portion in the open-seat District 6. A small corner of the county falls in [Renee Ellmers’s](https://ballotpedia.org/Renee_Ellmers) Republican District 2.

![Incumbency for county precincts](Alamance-incumbents.png)